In [57]:
import pandas as pd
import numpy as np
import warnings
from datetime import datetime
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [58]:
original_df = pd.read_csv("CountryTemperatures_byYear.csv").drop(columns=["Unnamed: 0"])
original_df.head()

,year,Country,continent,AverageTemperature,AverageTemperatureUncertainty,Latitude,Longitude,GHG,temp_diff
0,1900,AFGHANISTAN,Asia,13.749333,0.748833,33.00,65.0,0.0,0.0
1,1900,ALBANIA,Europe,13.068583,0.723417,41.00,20.0,0.0,0.0
2,1900,ALGERIA,Africa,22.864167,0.826667,28.00,3.0,0.0,0.0
3,1900,AMERICAN SAMOA,Oceania,26.273500,0.891000,-14.33,-170.0,NaN,0.0
4,1900,ANDORRA,Europe,11.348333,0.542167,42.50,1.5,0.0,0.0


In [59]:
original_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24342 entries, 0 to 24341
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   year                           24342 non-null  int64  
 1   Country                        24342 non-null  object 
 2   continent                      24342 non-null  object 
 3   AverageTemperature             24341 non-null  float64
 4   AverageTemperatureUncertainty  24341 non-null  float64
 5   Latitude                       24342 non-null  float64
 6   Longitude                      24342 non-null  float64
 7   GHG                            22458 non-null  float64
 8   temp_diff                      24341 non-null  float64
dtypes: float64(6), int64(1), object(2)
memory usage: 1.7+ MB


In [60]:
def get_temperature_diff_between(df_by_year,start_date,end_date):
    df = df_by_year.loc[(df_by_year["year"] >= start_date) & 
                        (df_by_year["year"] <= end_date)].groupby(["Country","continent"]).sum().reset_index()
    
    return df

In [61]:
df = get_temperature_diff_between(original_df,1950,2010)
df.head()

,Country,continent,year,AverageTemperature,AverageTemperatureUncertainty,Latitude,Longitude,GHG,temp_diff
0,AFGHANISTAN,Asia,120780,882.743167,26.149500,2013.00,3965.0,2.249394e+09,2.478583
1,ALBANIA,Europe,120780,793.627083,20.248083,2501.00,1220.0,6.573183e+09,1.257167
2,ALGERIA,Africa,120780,1431.353250,23.001000,1708.00,183.0,5.690670e+10,2.027500
3,AMERICAN SAMOA,Oceania,120780,1641.876083,23.710083,-874.13,-10370.0,0.000000e+00,0.602417
4,ANDORRA,Europe,120780,710.201500,18.835667,2592.50,91.5,1.068606e+08,-0.837083


In [64]:
fig = px.scatter(df, x="temp_diff", y="GHG", color="continent", template="simple_white", hover_name="Country",
                labels={
                     "temp_diff": "Increase in average temperature",
                     "GHG": "Greenhouse Gases Emissions"

                 },)
fig.show()

In [30]:
import plotly.graph_objs as go
import statsmodels.api as sm

fig = px.scatter(df, x="temp_diff", y ="GHG", size=np.log(df["GHG"]), color="temp_diff", hover_name=df.index, log_x=False,marginal_x = "box",marginal_y = "box", template="simple_white", color_discrete_sequence=["#0d0887", "#9c179e"])

# linear regression
regline = sm.OLS(df["AverageTemperature"],sm.add_constant(np.log(df["GHG"]))).fit().fittedvalues

# add linear regression line for whole sample
fig.add_traces(go.Scatter(x=np.log(df["GHG"]), y=regline,
                          mode = 'lines',
                          marker_color='#fb9f3a',
                          name='OLS Trendline')
                          )

fig.show()

C:\Users\asus\miniconda3\envs\data_vis\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning:

divide by zero encountered in log



ValueError: 
    Invalid element(s) received for the 'size' property of scatter.marker
        Invalid elements include: [-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf]

    The 'size' property is a number and may be specified as:
      - An int or float in the interval [0, inf]
      - A tuple, list, or one-dimensional numpy array of the above

In [45]:
df = df.dropna()
df= df[df['GHG'] != 0]
df

KeyError: 'GHG'

In [31]:
df = df.loc[(df['year'] == 1949)].groupby('Country')[['AverageTemperature', 'GHG']].mean().reset_index()

In [32]:
fig = px.scatter(df, x="AverageTemperature", y="GHG", color="Country", marginal_y="violin",
           marginal_x="box", trendline="ols", template="simple_white")
fig.show()

In [33]:
import plotly.graph_objs as go
import statsmodels.api as sm

col_name = str("AverageTemperature") + "(above Average)"
df[col_name] = (df[("AverageTemperature")] > df[("AverageTemperature")].mean())#.astype(int)

fig = px.scatter(df, x="AverageTemperature", y ="GHG", size=np.log(df["GHG"]), color=col_name, hover_name=df.index, log_x=False,marginal_x = "box",marginal_y = "box", template="simple_white", color_discrete_sequence=["#0d0887", "#9c179e"])

# linear regression
regline = sm.OLS(df["AverageTemperature"],sm.add_constant(np.log(df["GHG"]))).fit().fittedvalues

# add linear regression line for whole sample
fig.add_traces(go.Scatter(x=np.log(df["GHG"]), y=regline,
                          mode = 'lines',
                          marker_color='#fb9f3a',
                          name='OLS Trendline')
                          )

fig.show()

ValueError: zero-size array to reduction operation maximum which has no identity

In [26]:
col_name = str("AverageTemperature") + ": above Average"
df[col_name] = (df[("AverageTemperature")] > df[("AverageTemperature")].mean())#.astype(int)

    
size = df["AverageTemperature"]
    
fig = px.scatter(df, x="AverageTemperature", y ="GHG", size=size, color=col_name,hover_name=df.index, log_x=False, trendline = "ols",  trendline_color_override="#bd3786",marginal_x = "box",marginal_y = "box", template="simple_white", color_discrete_sequence=px.colors.qualitative.G10)
fig.show()

ValueError: 
    Invalid element(s) received for the 'size' property of scatter.marker
        Invalid elements include: [-4.948583333333334]

    The 'size' property is a number and may be specified as:
      - An int or float in the interval [0, inf]
      - A tuple, list, or one-dimensional numpy array of the above

In [27]:
def scatter_plot(df, x, y, size):
    '''insert x and y as string while x entails color coding (abv avg...)'''
    
    # feature engineering
    col_name = str(y) + " above avg"
    df[col_name] = (df[y] > df[y].mean()).astype(int)
    
    size = df[size] * 1/4
    
    # plotting
    fig = px.scatter(df, x=x, y =y, size=size, color_continuous_scale=px.colors.sequential.Plasma, color=col_name,hover_name=df.index, log_x=False, trendline = "ols",  trendline_color_override="#bd3786",marginal_x = "box",marginal_y = "box", template="simple_white")
    fig.show()

In [28]:
scatter_plot(df , "AverageTemperature", "GHG", "AverageTemperature")

ValueError: 
    Invalid element(s) received for the 'size' property of scatter.marker
        Invalid elements include: [-1.2371458333333334]

    The 'size' property is a number and may be specified as:
      - An int or float in the interval [0, inf]
      - A tuple, list, or one-dimensional numpy array of the above

In [29]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=df["Country"],
    y=np.log(df["GHG"]),
    name="GHG EMission",
    marker_color='#0d0887'
))
fig.add_trace(go.Bar(
    x=df["Country"],
    y=df["AverageTemperature"],
    name="Average Temperature",
    marker_color='#fdca26'
))

In [30]:
def get_temperature_diff_between(df_by_year,start_date,end_date):
    df = df_by_year.loc[(df_by_year["year"] >= start_date) & 
                        (df_by_year["year"] <= end_date)].groupby(["Country"]).sum()[["temp_diff"]]
    
    return df